In [1]:
import pandas as pd
from pathlib import Path
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from torch.utils.data import Subset
from torchcontrib.optim import SWA

In [2]:
Catholic_file = 'aug_train_v2.2.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,7
1,0002-2.wav,non-wheezing,0,1,7
2,0002-3.wav,non-wheezing,0,1,7
3,0002-4.wav,non-wheezing,0,1,7
4,0003-2.wav,non-wheezing,0,0,1
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,9
1606,wn_0588-1.wav,wheezing,1,0,1
1607,wn_0599-3.wav,wheezing,1,1,6
1608,wn_0602-2.wav,wheezing,1,1,8


In [3]:
df = df.sample(frac=1)

In [4]:
df

,filename,category,class,sex,old
1431,wn_0021-4.wav,non-wheezing,0,0,0
1180,ts_0013-1.wav,non-wheezing,0,0,2
939,rs_0009-3.wav,non-wheezing,0,0,3
65,0027-2.wav,non-wheezing,0,0,1
217,0025-1.wav,wheezing,1,1,0
...,...,...,...,...,...
1548,wn_0534-4.wav,wheezing,1,0,9
463,cs_0002-4.wav,non-wheezing,0,1,7
614,cs_0107-1.wav,wheezing,1,0,8
1563,wn_0581-2.wav,wheezing,1,1,7


In [5]:
mean = df.old.mean()
std = df.old.std()
df.old = (df.old - mean) / std

In [6]:
df

,filename,category,class,sex,old
1431,wn_0021-4.wav,non-wheezing,0,0,-1.257097
1180,ts_0013-1.wav,non-wheezing,0,0,-0.639293
939,rs_0009-3.wav,non-wheezing,0,0,-0.330391
65,0027-2.wav,non-wheezing,0,0,-0.948195
217,0025-1.wav,wheezing,1,1,-1.257097
...,...,...,...,...,...
1548,wn_0534-4.wav,wheezing,1,0,1.523021
463,cs_0002-4.wav,non-wheezing,0,1,0.905217
614,cs_0107-1.wav,wheezing,1,0,1.214119
1563,wn_0581-2.wav,wheezing,1,1,0.905217


In [7]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class', 'sex', 'old']]
df.head()

,relative_path,class,sex,old
1431,/wn_0021-4.wav,0,0,-1.257097
1180,/ts_0013-1.wav,0,0,-0.639293
939,/rs_0009-3.wav,0,0,-0.330391
65,/0027-2.wav,0,0,-0.948195
217,/0025-1.wav,1,1,-1.257097


In [8]:
data_path = 'aug_train_v2.2'

In [ ]:
pd

In [9]:
class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def spectro_gram(aud, n_mels=64, n_fft=2048, hop_len=160):
    sig,sr = aud
    top_db = 80
    
    spec = torchaudio.transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def mfcc(aud, n_mfcc=64, melkwargs={"n_fft": 2048, "hop_length": 160, "power": 2}): #melkwargs={"n_fft": 2048, "hop_length": 512, "power": 2} #n_mfcc=64, n_fft=2048, hop_len=512
    sig,sr = aud
    top_db = 80
    mfcc = torchaudio.transforms.MFCC(sr, n_mfcc=64, melkwargs={"n_fft": 2048, "hop_length": 160, "power": 2})(sig)
    mfcc = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(mfcc)
    return (mfcc)
    
  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = torchaudio.transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = torchaudio.transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)
    
  def mfcc_augment(mfcc, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = mfcc.shape
    mask_value = mfcc.mean()
    aug_mfcc = mfcc

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = torchaudio.transforms.FrequencyMasking(freq_mask_param)(aug_mfcc, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = torchaudio.transforms.TimeMasking(time_mask_param)(aug_mfcc, mask_value)

    return aug_mfcc
    print(aug_mfcc)

In [10]:
class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 22050
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    mfcc = Breath_sound_Util.mfcc(dur_aud, n_mfcc=64, melkwargs={"n_fft": 1024, "hop_length": None, "power": 2})
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    aug_mfcc = Breath_sound_Util.mfcc_augment(mfcc, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df.loc[idx, 'old']
    y = self.df.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    aug_sgram = torch.cat((aug_sgram, aug_mfcc), 0)
    
    return aug_sgram, tabular, class_id

In [11]:
brds = breathDS(df, data_path)

In [12]:
train_dl = DataLoader(brds, batch_size=32, shuffle=True)

In [13]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [14]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc_tab = nn.Linear(2, 8)
        self.fc_tab2 = nn.Linear(8, 16)
        self.fc = nn.Linear((512 * block.expansion)+16, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x, x2):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x2 = self.fc_tab(x2)
        x2 = self.fc_tab2(x2)
        x = torch.cat((x, x2), 1)
        x = self.fc(x)

        return x

In [15]:
Model1 = ResNet(BasicBlock, [3, 4, 6, 3])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

device(type='cuda', index=0)

In [16]:
def training(model, train_dl, num_epochs):
  criterion = nn.CrossEntropyLoss()
  #optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  base_optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
  optimizer = SWA(base_optimizer, swa_start=10, swa_freq=5, swa_lr=0.001)
  #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,
                                                ##steps_per_epoch=int(len(train_dl)),
                                                #epochs=num_epochs,
                                                #anneal_strategy='linear')

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0
    correct_prediction_1 = 0
    total_prediction_1 = 0

    for i, data in enumerate(train_dl):
        x, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        #inputs, inputs2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        #여기에도 inputs2를 추가시켜줫음. 이제 data[1]이 가리키는건 breathDS의 tablular임
        #x_m, x_s = x.mean(), x.std()
        #x = (x - x_m) / x_s
        optimizer.zero_grad()
        output = model(x, x2)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        running_loss += loss.item()
        _, prediction = torch.max(output,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]


    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    #val_acc = correct_prediction_1/total_prediction_1
    print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')

  print('Finished Training')

In [17]:
num_epochs=60
training(Model1, train_dl, num_epochs)

Epoch: 0, Loss: 0.4150, Accuracy: 0.8410
Epoch: 1, Loss: 0.3565, Accuracy: 0.8602
Epoch: 2, Loss: 0.3225, Accuracy: 0.8714
Epoch: 3, Loss: 0.2786, Accuracy: 0.8876
Epoch: 4, Loss: 0.2400, Accuracy: 0.9031
Epoch: 5, Loss: 0.1372, Accuracy: 0.9534
Epoch: 6, Loss: 0.1707, Accuracy: 0.9354
Epoch: 7, Loss: 0.0809, Accuracy: 0.9714
Epoch: 8, Loss: 0.0755, Accuracy: 0.9683
Epoch: 9, Loss: 0.0571, Accuracy: 0.9814
Epoch: 10, Loss: 0.0479, Accuracy: 0.9807
Epoch: 11, Loss: 0.0646, Accuracy: 0.9801
Epoch: 12, Loss: 0.0495, Accuracy: 0.9832
Epoch: 13, Loss: 0.0134, Accuracy: 0.9957
Epoch: 14, Loss: 0.0028, Accuracy: 0.9994
Epoch: 15, Loss: 0.0015, Accuracy: 1.0000
Epoch: 16, Loss: 0.0008, Accuracy: 1.0000
Epoch: 17, Loss: 0.0004, Accuracy: 1.0000
Epoch: 18, Loss: 0.0002, Accuracy: 1.0000
Epoch: 19, Loss: 0.0001, Accuracy: 1.0000
Epoch: 20, Loss: 0.0001, Accuracy: 1.0000
Epoch: 21, Loss: 0.0001, Accuracy: 1.0000
Epoch: 22, Loss: 0.0007, Accuracy: 1.0000
Epoch: 23, Loss: 0.0843, Accuracy: 0.9739
Ep

In [21]:
Catholic_file1 = 'test_v2.2.csv'
df1 = pd.read_csv(Catholic_file1)
df1

,filename,category,class,sex,old
0,0001-2.wav,non-wheezing,0,0,0
1,0003-1.wav,non-wheezing,0,0,1
2,0004-1.wav,non-wheezing,0,0,1
3,0004-4.wav,non-wheezing,0,0,1
4,0006-1.wav,non-wheezing,0,1,1
5,0007-1.wav,non-wheezing,0,0,7
6,0008-1.wav,non-wheezing,0,0,2
7,0009-1.wav,non-wheezing,0,0,3
8,0010-3.wav,non-wheezing,0,1,3
9,0012-1.wav,non-wheezing,0,0,2


In [22]:
mean = df1.old.mean()
std = df1.old.std()
df1.old = (df1.old - mean) / std

In [23]:
df1['relative_path'] = '/' + df1['filename'].astype(str)
df1 = df1[['relative_path', 'class', 'sex', 'old']]
df1.head()

,relative_path,class,sex,old
0,/0001-2.wav,0,0,-1.131772
1,/0003-1.wav,0,0,-0.809217
2,/0004-1.wav,0,0,-0.809217
3,/0004-4.wav,0,0,-0.809217
4,/0006-1.wav,0,1,-0.809217


In [24]:
data_path = 'test_v2.2'

In [25]:
class Breath_sound_Util1():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)

  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def mfcc(aud, n_mfcc=64, melkwargs={"n_fft": 1024, "hop_length": None, "power": 2}): #melkwargs={"n_fft": 2048, "hop_length": 512, "power": 2} #n_mfcc=64, n_fft=2048, hop_len=512
    sig,sr = aud
    top_db = 80
    mfcc = torchaudio.transforms.MFCC(sr, n_mfcc=64, melkwargs={"n_fft": 1024, "hop_length": None, "power": 2})(sig)
    mfcc = torchaudio.transforms.AmplitudeToDB(top_db=top_db)(mfcc)
    return (mfcc)

In [26]:
class breathDS1(Dataset):
    
  def __init__(self, df1, data_path):
    self.df1 = df1
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df1)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df1.loc[idx, 'relative_path']
    class_id = self.df1.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    mfcc = Breath_sound_Util.mfcc(dur_aud, n_mfcc=64, melkwargs={"n_fft": 1024, "hop_length": None, "power": 2})
    #aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df1.loc[idx, 'old']
    y = self.df1.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    aug_sgram = torch.cat((sgram, mfcc), 0)
    
    return aug_sgram, tabular, class_id

In [27]:
brds1 = breathDS1(df1, data_path)

In [28]:
test_dl = torch.utils.data.DataLoader(brds1, batch_size=32, shuffle=True)

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score

def inference (model, test_dl):
  correct_prediction = 0
  total_prediction = 0
  TP = 0 
  TN = 0
  FN = 0
  FP = 0
  a = []
  b = []
  with torch.no_grad():
    for i, data in enumerate(test_dl):
      
      x, x2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
      cpu_labels = data[2]
      x_m, x_s = x.mean(), x.std()
      x = (x - x_m) / x_s
      outputs = model(x, x2)
 
      _, prediction = torch.max(outputs,1)
      
      cpu_prediction = prediction.to('cpu')
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
          

      TP += ((prediction == 1) & (labels == 1)).sum()
# TN predict and label are both 0
      TN += ((prediction == 0) & (labels == 0)).sum()
# FN    predict 0 label 1
      FN += ((prediction == 0) & (labels == 1)).sum()
# FP    predict 1 label 0
      FP += ((prediction == 1) & (labels == 0)).sum()

      c = cpu_labels.tolist()
      a.extend(c)

      d = cpu_prediction.tolist()
      b.extend(d)
    
    # TP : 휘징인데 휘징이라고 판단(잘함)
    # FP : 헬시인데 휘징이라고 판단(못함)
    # TN : 헬시인데 헬시라고 판단(잘함)
    # FN : 휘징인데 헬시라고 판단(못함)
    
    # POSITIVE : 1(휘징)이라고 판단
    # NEGATIVE : 0(헬시)라고 판단
    # TRUE : 정답값 1(휘징)
    # FALSE : 정답값 0(헬시)
    
    # PRECISION : 휘징이라고 판단한 것 중 진짜 휘징
    # RECALL : 휘징인데 휘징이라고 잘 판단
    

  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  F1 = 2 * recall * precision / (recall + precision)
  print(TP)
  print(FP)
  print(FN)
  print(TN)

  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.4f}, Total items: {total_prediction}')
  print(f'precision: {precision:.4f}, F1: {F1:.4f}')
  print(f'recall: {recall:.4f}')
  target_names = ['healthy', 'wheezing']
  print(classification_report(a, b, target_names = target_names))
  print("AUC:{}".format(roc_auc_score(a, b)))

inference(Model1, test_dl)

tensor(15, device='cuda:0')
tensor(4, device='cuda:0')
tensor(6, device='cuda:0')
tensor(32, device='cuda:0')
Accuracy: 0.8246, Total items: 57
precision: 0.7895, F1: 0.7500
recall: 0.7143
              precision    recall  f1-score   support

     healthy       0.84      0.89      0.86        36
    wheezing       0.79      0.71      0.75        21

    accuracy                           0.82        57
   macro avg       0.82      0.80      0.81        57
weighted avg       0.82      0.82      0.82        57

AUC:0.8015873015873016


In [75]:
import torch
import torch.onnx as onnx
import torchvision.models as models

torch.save(Model1.state_dict(), 'best_resnet18.pth')